In [1]:
import sys
    # caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'D:/books/python/0.   Fluent Python, 2nd Edition/example-code-2e/17-it-generator/')
sys.path

['D:\\books\\python\\0.   Fluent Python, 2nd Edition',
 'D:/books/python/0.   Fluent Python, 2nd Edition/example-code-2e/17-it-generator/',
 'C:\\Users\\lidan\\miniconda3\\python38.zip',
 'C:\\Users\\lidan\\miniconda3\\DLLs',
 'C:\\Users\\lidan\\miniconda3\\lib',
 'C:\\Users\\lidan\\miniconda3',
 '',
 'C:\\Users\\lidan\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\magic_impute-2.0.4-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\seqc-0.2.0-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\weasyprint-56.1-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\cairocffi-1.3.0-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\win32',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\Pythonwin']

In [2]:
"""
Sentence: access words by index

    >>> text = 'To be, or not to be, that is the question'
    >>> s = Sentence(text)
    >>> len(s)
    10
    >>> s[1], s[5]
    ('be', 'be')
    >>> s
    Sentence('To be, or no... the question')

"""

# tag::SENTENCE_SEQ[]
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)  # <1>

    def __getitem__(self, index):
        return self.words[index]  # <2>

    def __len__(self):  # <3>
        return len(self.words)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)  # <4>

# end::SENTENCE_SEQ[]


In [3]:
help(re.compile)

Help on function compile in module re:

compile(pattern, flags=0)
    Compile a regular expression pattern, returning a Pattern object.



In [4]:
help(re.compile(r'\w+').findall)

Help on built-in function findall:

findall(string, pos=0, endpos=9223372036854775807) method of re.Pattern instance
    Return a list of all non-overlapping matches of pattern in string.



Regarding the specific regex expression, this searches for groups that have alphanumerics (that's the `\w` part) or apostrophes (which is also in those square brackets) that are 1 or longer. Note that whitespace is not a match, so this, generally speaking, breaks a line into words.

In [5]:
text = 'To be, or not to be, that is the question'
s = Sentence(text)
len(s)

10

In [6]:
s[1], s[5]

('be', 'be')

In [7]:
s

Sentence('To be, or no... the question')

In [8]:
"""
Sentence: iterate over words using the Iterator Pattern, take #1

WARNING: the Iterator Pattern is much simpler in idiomatic Python;
see: sentence_gen*.py.
"""

# tag::SENTENCE_ITER[]
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):  # <1>
        return SentenceIterator(self.words)  # <2>


class SentenceIterator:

    def __init__(self, words):
        self.words = words  # <3>
        self.index = 0  # <4>

    def __next__(self):
        try:
            word = self.words[self.index]  # <5>
        except IndexError:
            raise StopIteration()  # <6>
        self.index += 1  # <7>
        return word  # <8>

    def __iter__(self):  # <9>
        return self
# end::SENTENCE_ITER[]


In [9]:
"""
Sentence: iterate over words using the Iterator Pattern, take #2

WARNING: the Iterator Pattern is much simpler in idiomatic Python;
see: sentence_gen*.py.
"""

import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        word_iter = RE_WORD.finditer(self.text)  # <1>
        return SentenceIter(word_iter)  # <2>


class SentenceIter:

    def __init__(self, word_iter):
        self.word_iter = word_iter  # <3>

    def __next__(self):
        match = next(self.word_iter)  # <4>
        return match.group()  # <5>

    def __iter__(self):
        return self


In [10]:
text = 'To be, or not to be, that is the question'
s = Sentence(text)
s

Sentence('To be, or no... the question')

In [11]:
print(s)

Sentence('To be, or no... the question')


In [12]:
for t in s:
    print(t)

To
be
or
not
to
be
that
is
the
question


In [13]:
"""
Sentence: iterate over words using a generator function
"""

# tag::SENTENCE_GEN2[]
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text  # <1>

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):  # <2>
            yield match.group()  # <3>

# end::SENTENCE_GEN2[]


In [14]:
text = 'To be, or not to be, that is the question'
s = Sentence(text)
s

Sentence('To be, or no... the question')

In [15]:
print(s)

Sentence('To be, or no... the question')


In [16]:
for t in s:
    print(t)

To
be
or
not
to
be
that
is
the
question


In [17]:
for match in re.compile(r'\w+').finditer(text):
    print(match)

<re.Match object; span=(0, 2), match='To'>
<re.Match object; span=(3, 5), match='be'>
<re.Match object; span=(7, 9), match='or'>
<re.Match object; span=(10, 13), match='not'>
<re.Match object; span=(14, 16), match='to'>
<re.Match object; span=(17, 19), match='be'>
<re.Match object; span=(21, 25), match='that'>
<re.Match object; span=(26, 28), match='is'>
<re.Match object; span=(29, 32), match='the'>
<re.Match object; span=(33, 41), match='question'>


In [18]:
for match in re.compile(r'\w+').finditer(text):
    print(match.group())

To
be
or
not
to
be
that
is
the
question


In [19]:
"""
Sentence: iterate over words using a generator expression
"""

# tag::SENTENCE_GENEXP[]
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))
# end::SENTENCE_GENEXP[]


In [20]:
text = 'To be, or not to be, that is the question'
s = Sentence(text)
s

Sentence('To be, or no... the question')

In [21]:
print(s)

Sentence('To be, or no... the question')


In [22]:
for t in s:
    print(t)

To
be
or
not
to
be
that
is
the
question


In [23]:
(match.group() for match in RE_WORD.finditer(text))

<generator object <genexpr> at 0x0000019C58A88820>

In [24]:
for t in (match.group() for match in RE_WORD.finditer(text)):
    print(t)

To
be
or
not
to
be
that
is
the
question


In [25]:
"""
Arithmetic progression class

    >>> ap = ArithmeticProgression(1, .5, 3)
    >>> list(ap)
    [1.0, 1.5, 2.0, 2.5]


"""


class ArithmeticProgression:

    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end  # None -> "infinite" series

    def __iter__(self):
        result_type = type(self.begin + self.step)
        result = result_type(self.begin)
        forever = self.end is None
        while forever or result < self.end:
            yield result
            result += self.step


In [26]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [27]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [28]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [29]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [30]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [31]:
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

In [32]:
"""
Arithmetic progression class

# tag::ARITPROG_CLASS_DEMO[]

    >>> ap = ArithmeticProgression(0, 1, 3)
    >>> list(ap)
    [0, 1, 2]
    >>> ap = ArithmeticProgression(1, .5, 3)
    >>> list(ap)
    [1.0, 1.5, 2.0, 2.5]
    >>> ap = ArithmeticProgression(0, 1/3, 1)
    >>> list(ap)
    [0.0, 0.3333333333333333, 0.6666666666666666]
    >>> from fractions import Fraction
    >>> ap = ArithmeticProgression(0, Fraction(1, 3), 1)
    >>> list(ap)
    [Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]
    >>> from decimal import Decimal
    >>> ap = ArithmeticProgression(0, Decimal('.1'), .3)
    >>> list(ap)
    [Decimal('0'), Decimal('0.1'), Decimal('0.2')]

# end::ARITPROG_CLASS_DEMO[]
"""


# tag::ARITPROG_CLASS[]
class ArithmeticProgression:

    def __init__(self, begin, step, end=None):       # <1>
        self.begin = begin
        self.step = step
        self.end = end  # None -> "infinite" series

    def __iter__(self):
        result_type = type(self.begin + self.step)   # <2>
        result = result_type(self.begin)             # <3>
        forever = self.end is None                   # <4>
        index = 0
        while forever or result < self.end:          # <5>
            yield result                             # <6>
            index += 1
            result = self.begin + self.step * index  # <7>
# end::ARITPROG_CLASS[]


In [33]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [34]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [35]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [36]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [37]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [38]:
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

In [39]:
"""
Arithmetic progression generator function::

    >>> ap = aritprog_gen(1, .5, 3)
    >>> list(ap)
    [1.0, 1.5, 2.0, 2.5]
    >>> ap = aritprog_gen(0, 1/3, 1)
    >>> list(ap)
    [0.0, 0.3333333333333333, 0.6666666666666666]
    >>> from fractions import Fraction
    >>> ap = aritprog_gen(0, Fraction(1, 3), 1)
    >>> list(ap)
    [Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]
    >>> from decimal import Decimal
    >>> ap = aritprog_gen(0, Decimal('.1'), .3)
    >>> list(ap)
    [Decimal('0'), Decimal('0.1'), Decimal('0.2')]

"""


# tag::ARITPROG_GENFUNC[]
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index
# end::ARITPROG_GENFUNC[]


In [40]:
ap = aritprog_gen(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [41]:
ap = aritprog_gen(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [42]:
from fractions import Fraction
ap = aritprog_gen(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [43]:
from decimal import Decimal
ap = aritprog_gen(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

In [44]:
import itertools
gen = itertools.count(1, .5)
next(gen)

1

In [45]:
itertools.count(1, .5)

count(1, 0.5)

In [46]:
next(gen)

1.5

In [47]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [48]:
# tag::ARITPROG_ITERTOOLS[]
import itertools


def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is None:
        return ap_gen
    return itertools.takewhile(lambda n: n < end, ap_gen)
# end::ARITPROG_ITERTOOLS[]


In [49]:
def vowel(c):
    return c.lower() in 'aeiou'

In [50]:
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [51]:
import itertools
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

The `drop-while` would stop dropping when reaching "r" which is not a vowel, and take the rest, and the `take-while` would stop taking when reaching "r" which is not a vowel, and drop the rest.

In [52]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [53]:
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

In [54]:
list(itertools.compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [55]:
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [56]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [57]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
import itertools
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [58]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [59]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [60]:
import operator
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [61]:
list(itertools.accumulate(range(1, 11), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [62]:
def tree(cls):
    yield cls.__name__


if __name__ == '__main__':
    for cls_name in tree(BaseException):
        print(cls_name)


BaseException


In [63]:
BaseException.__name__

'BaseException'

In [64]:
BaseException.__subclasses__()

[Exception,
 GeneratorExit,
 SystemExit,
 KeyboardInterrupt,
 asyncio.exceptions.CancelledError,
 _pydev_imps._pydev_saved_modules.DebuggerInitializationError,
 pkg_resources._vendor.more_itertools.more.AbortThread,
 setuptools._vendor.more_itertools.more.AbortThread]

In [65]:
Exception.__subclasses__()

[TypeError,
 StopAsyncIteration,
 StopIteration,
 ImportError,
 OSError,
 EOFError,
 RuntimeError,
 NameError,
 AttributeError,
 SyntaxError,
 LookupError,
 ValueError,
 AssertionError,
 ArithmeticError,
 SystemError,
 ReferenceError,
 MemoryError,
 BufferError,
 Warning,
 locale.Error,
 warnings._OptionError,
 runpy._Error,
 re.error,
 sre_parse.Verbose,
 subprocess.SubprocessError,
 tokenize.TokenError,
 tokenize.StopTokenizing,
 inspect.EndOfBlock,
 traitlets.traitlets.TraitError,
 copy.Error,
 concurrent.futures._base.Error,
 socket._GiveupOnSendfile,
 struct.error,
 binascii.Incomplete,
 asyncio.exceptions.TimeoutError,
 asyncio.exceptions.InvalidStateError,
 asyncio.exceptions.LimitOverrunError,
 asyncio.queues.QueueEmpty,
 asyncio.queues.QueueFull,
 zlib.error,
 _lzma.LZMAError,
 shutil.RegistryError,
 shutil._GiveupOnFastCopy,
 _queue.Empty,
 queue.Full,
 zmq.error.ZMQBaseError,
 _pickle.PickleError,
 pickle._Stop,
 argparse.ArgumentError,
 argparse.ArgumentTypeError,
 traitlet

In [66]:
def tree(cls):
    yield cls.__name__, 0
    for sub_cls in cls.__subclasses__():
        yield sub_cls.__name__, 1


if __name__ == '__main__':
    for cls_name, level in tree(BaseException):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')


BaseException
    Exception
    GeneratorExit
    SystemExit
    KeyboardInterrupt
    CancelledError
    DebuggerInitializationError
    AbortThread
    AbortThread


In [67]:
def tree(cls):
    yield cls.__name__, 0
    for sub_cls in cls.__subclasses__():
        yield sub_cls.__name__, 1
        for sub_sub_cls in sub_cls.__subclasses__():
            yield sub_sub_cls.__name__, 2


if __name__ == '__main__':
    for cls_name, level in tree(BaseException):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')


BaseException
    Exception
        TypeError
        StopAsyncIteration
        StopIteration
        ImportError
        OSError
        EOFError
        RuntimeError
        NameError
        AttributeError
        SyntaxError
        LookupError
        ValueError
        AssertionError
        ArithmeticError
        SystemError
        ReferenceError
        MemoryError
        BufferError
        Warning
        Error
        _OptionError
        _Error
        error
        Verbose
        SubprocessError
        TokenError
        StopTokenizing
        EndOfBlock
        TraitError
        Error
        Error
        _GiveupOnSendfile
        error
        Incomplete
        TimeoutError
        InvalidStateError
        LimitOverrunError
        QueueEmpty
        QueueFull
        error
        LZMAError
        RegistryError
        _GiveupOnFastCopy
        Empty
        Full
        ZMQBaseError
        PickleError
        _Stop
        ArgumentError
        ArgumentType

In [68]:
GeneratorExit.__subclasses__()

[]

In [69]:
def tree(cls, level=0):
    yield cls.__name__, level
    for sub_cls in cls.__subclasses__():
        yield from tree(sub_cls, level + 1)


if __name__ == '__main__':
    for cls_name, level in tree(BaseException):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')


BaseException
    Exception
        TypeError
            FloatOperation
            MultipartConversionError
        StopAsyncIteration
        StopIteration
        ImportError
            ModuleNotFoundError
                PackageNotFoundError
                PackageNotFoundError
            ZipImportError
        OSError
            ConnectionError
                BrokenPipeError
                ConnectionAbortedError
                ConnectionRefusedError
                ConnectionResetError
                    RemoteDisconnected
            BlockingIOError
            ChildProcessError
            FileExistsError
            FileNotFoundError
            IsADirectoryError
            NotADirectoryError
            InterruptedError
                InterruptedSystemCall
            PermissionError
            ProcessLookupError
            TimeoutError
            UnsupportedOperation
            herror
            gaierror
            timeout
            SSLError
                

In [72]:
"""
A coroutine to compute a running average

# tag::CORO_AVERAGER_TEST[]
    >>> coro_avg = averager()  # <1>
    >>> next(coro_avg)  # <2>
    0.0
    >>> coro_avg.send(10)  # <3>
    10.0
    >>> coro_avg.send(30)
    20.0
    >>> coro_avg.send(5)
    15.0

# end::CORO_AVERAGER_TEST[]
# tag::CORO_AVERAGER_TEST_CONT[]

    >>> coro_avg.send(20)  # <1>
    16.25
    >>> coro_avg.close()  # <2>
    >>> coro_avg.close()  # <3>
    >>> coro_avg.send(5)  # <4>
    Traceback (most recent call last):
      ...
    StopIteration

# end::CORO_AVERAGER_TEST_CONT[]

"""

# tag::CORO_AVERAGER[]
from collections.abc import Generator

def averager() -> Generator:  # <1>
    total = 0.0
    count = 0
    average = 0.0
    while True:  # <2>
        term = yield average  # <3>
        total += term
        count += 1
        average = total/count
# end::CORO_AVERAGER[]


In [76]:
coro_avg = averager()  # <1>
next(coro_avg) 

0.0

In [77]:
coro_avg.send(10)

10.0

In [78]:
coro_avg.send(30)

20.0

In [79]:
coro_avg.send(5)

15.0

In [80]:
coro_avg.send(20)

16.25

In [161]:
"""
A coroutine to compute a running average.

Testing ``averager2`` by itself::

# tag::RETURNING_AVERAGER_DEMO_1[]

    >>> coro_avg = averager2()
    >>> next(coro_avg)
    >>> coro_avg.send(10)  # <1>
    >>> coro_avg.send(30)
    >>> coro_avg.send(6.5)
    >>> coro_avg.close()  # <2>

# end::RETURNING_AVERAGER_DEMO_1[]

Catching `StopIteration` to extract the value returned by
the coroutine::

# tag::RETURNING_AVERAGER_DEMO_2[]

    >>> coro_avg = averager2()
    >>> next(coro_avg)
    >>> coro_avg.send(10)
    >>> coro_avg.send(30)
    >>> coro_avg.send(6.5)
    >>> try:
    ...     coro_avg.send(STOP)  # <1>
    ... except StopIteration as exc:
    ...     result = exc.value  # <2>
    ...
    >>> result  # <3>
    Result(count=3, average=15.5)

# end::RETURNING_AVERAGER_DEMO_2[]

Using `yield from`:


# tag::RETURNING_AVERAGER_DEMO_3[]

    >>> def compute():
    ...     res = yield from averager2(True)  # <1>
    ...     print('computed:', res)  # <2>
    ...     return res  # <3>
    ...
    >>> comp = compute()  # <4>
    >>> for v in [None, 10, 20, 30, STOP]:  # <5>
    ...     try:
    ...         comp.send(v)  # <6>
    ...     except StopIteration as exc:  # <7>
    ...         result = exc.value
    received: 10
    received: 20
    received: 30
    received: <Sentinel>
    computed: Result(count=3, average=20.0)
    >>> result  # <8>
    Result(count=3, average=20.0)

# end::RETURNING_AVERAGER_DEMO_3[]
"""

# tag::RETURNING_AVERAGER_TOP[]
from collections.abc import Generator
from typing import Union, NamedTuple

class Result(NamedTuple):  # <1>
    count: int  # type: ignore  # <2>
    average: float

class Sentinel:  # <3>
    def __repr__(self):
        return f'<Sentinel>'

STOP = Sentinel()  # <4>

SendType = Union[float, Sentinel]  # <5>
# end::RETURNING_AVERAGER_TOP[]
# tag::RETURNING_AVERAGER[]
def averager2(verbose: bool = False) -> Generator:  # <1>
    total = 0.0
    count = 0
    average = 0.0
    while True:
        term = yield  # <2>
        if verbose:
            print('received:', term)
        if isinstance(term, Sentinel):  # <3>
            break
        total += term  # <4>
        count += 1
        average = total / count
    return Result(count, average)  # <5>

# end::RETURNING_AVERAGER[]


In [162]:
def compute(): 
    res = yield from averager2(True)  # <1>
    print('computed:', res)  # <2>
    return res  # <3>

In [163]:
comp = compute()

In [164]:
for v in [None, 10, 20, 30, STOP]:  # <5>
    try:
        comp.send(v)  # <6>
        
    except StopIteration as exc:  # <7>
        result = exc.value

received: 10
received: 20
received: 30
received: <Sentinel>
computed: Result(count=3, average=20.0)


In [131]:
def compute(): 
    res = yield from averager2(True)  # <1>
    print('computed:', res)  # <2>
    return res  # <3>
comp = compute()

In [132]:
for v in [None, 10, 20, 30, STOP]:  # <5>
    try:
        comp.send(v)  # <6>
        
    except StopIteration as exc:  # <7>
        result = exc.value
        print(result)
        print(result)

received: 10
received: 20
received: 30
received: <Sentinel>
computed: Result(count=3, average=20.0)
Result(count=3, average=20.0)
Result(count=3, average=20.0)


In [126]:
StopIteration.value

<member 'value' of 'StopIteration' objects>

In [124]:
def compute(): 
    res = yield from averager2(True)  # <1>
    print('computed:', res)  # <2>
    return res  # <3>
comp = compute()

In [125]:
for v in [None, 10, 20, STOP, 30]:  # <5>
    try:
        comp.send(v)  # <6>
        
    except StopIteration as exc:  # <7>
        result = exc.value

received: 10
received: 20
received: <Sentinel>
computed: Result(count=2, average=15.0)


In [135]:
def compute(): 
    res = yield from averager2(True)  # <1>
    print('computed:', res)  # <2>
    return res  # <3>
comp = compute()

Must send `None` first

In [136]:
for v in [10, 20, STOP, 30]:  # <5>
    try:
        comp.send(v)  # <6>
        
    except StopIteration as exc:  # <7>
        result = exc.value

TypeError: can't send non-None value to a just-started generator

In [165]:
def compute(): 
    res = yield from averager2(True)  # <1>
    print('computed:', res)  # <2>
    return res  # <3>
comp = compute()

In [166]:
comp.send(None)
comp.send(100)
comp.send(500)
comp.send(STOP)

received: 100
received: 500
received: <Sentinel>
computed: Result(count=2, average=300.0)


StopIteration: Result(count=2, average=300.0)

Must send `STOP`

In [169]:
def compute(): 
    res = yield from averager2(True)  # <1>
    print('computed:', res)  # <2>
    return res  # <3>
comp = compute()

In [170]:
for v in [None, 10, 20, HELLO, 30]:  # <5>
    try:
        comp.send(v)  # <6>
        
    except StopIteration as exc:  # <7>
        result = exc.value

NameError: name 'HELLO' is not defined

In [171]:
coro_avg = averager2()  # <1>
next(coro_avg) 

In [172]:
coro_avg.send(10)

In [173]:
coro_avg.send(30)

In [174]:
coro_avg.send(5)

In [175]:
coro_avg.send(20)

In [176]:
try:
    coro_avg.send(STOP)  # <6>
except StopIteration as exc:  # <7>
    result = exc.value

In [177]:
result

Result(count=4, average=16.25)

In [155]:
help(Generator)

Help on class Generator in module collections.abc:

class Generator(Iterator)
 |  Method resolution order:
 |      Generator
 |      Iterator
 |      Iterable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __next__(self)
 |      Return the next item from the generator.
 |      When exhausted, raise StopIteration.
 |  
 |  close(self)
 |      Raise GeneratorExit inside generator.
 |  
 |  send(self, value)
 |      Send a value into the generator.
 |      Return next yielded value or raise StopIteration.
 |  
 |  throw(self, typ, val=None, tb=None)
 |      Raise an exception in the generator.
 |      Return next yielded value or raise StopIteration.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __subclasshook__(C) from abc.ABCMeta
 |      Abstract classes can override this to customize issubclass().
 |      
 |      This is invoked early on by abc.ABCMeta.__subclasscheck__().
 |      It should ret